**кластеризатор текстов**

SnowballStemmer + TFIDF + DBSCAN

_Евгений Борисов <esborisov@sevsu.ru>_

## тексты

In [1]:
import pandas as pd
pd.options.display.precision = 2 
pd.options.display.max_colwidth = 1000 

from tqdm.notebook import tqdm
tqdm.pandas()

датасеты для примеров  https://disk.yandex.ru/d/3_WAa7SgrQYBzw

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(2)

записей: 3196


,text,tag
1499,"Народный депутат Надежда Савченко не согласовывает свои действия и поступки с фракцией и партией «Батьківщина». Об этом на брифинге сообщила лидер политического объединения Юлия Тимошенко. Так она отреагировала на встречу своей коллеги с лидерами самопровозглашенных Донецкой и Луганской народных республик.\n\n\n\nПо словам лидера «Батьківщины», Надежда Савченко намерена создать свое собственное политическое объединение. «Поэтому она не координирует свои действия с партией ""Батьківщина"". Поэтому ""Батьківщина"" не несет политической ответственности за эти действия», - пояснила Тимошенко.\n\n\n\nНапомним, сегодня стало известно, что Надежда Савченко тайно встретилась в Минске с лидерами Донбасса Александром Захарченко и Игорем Плотницким. Народный депутат оценила прошедшие переговоры, как действенные, добавив, что она увидела в главах ЛНР и ДНР «людей, а не чертей».\n\nВидео: 112 Украина/YouTube\n\nФото: Maxym Marusenko / ZUMAPRESS / globallookpress",politics
253,Акция в поддержку Музея кино проходит сегодня вечером в Москве В знак\nпротеста против закрытия музея участники акции подходят ко входу в\nкинотеатр и в определенном месте оставляют камень.,culture


In [3]:
len( data.drop_duplicates('text') )

3196

## токенайзер

In [4]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word

# def tokenizer(text):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#         ]

In [5]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word
# from nltk.corpus import stopwords as nltk_stopwords

# stopwords = set(nltk_stopwords.words('russian'))

# def tokenizer(text,stopwords=stopwords):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#             if not (
#                False
#                or (len(t)<3) # выкидываем очень короткие слова
#                or re.match(r'^[^a-zA-ZЁёА-я]+$', t) # выкидываем токены не содержащие букв
#                or re.match(r'^(\w)\1+$', t)  # выкидываем токены из одного повторяющегося символа
#                or re.match(r'^[^a-zA-ZЁёА-я].*$', t)  # выкидываем токены начинающиеся не с буквы
#                or (t in stopwords) # выкидываем предлоги, союзы и т.п.    
#             )
#         ] 

In [6]:
import re
# from razdel import sentenize
from razdel import tokenize
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('russian'))

def tokenizer(text,stopwords=stopwords):
    return [
            t.text for t in tokenize( # разбиваем текст на слова
                re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
            ) 
            if not (
               False
               or (len(t.text)<3) # выкидываем очень короткие слова
               or re.match(r'^[^a-zA-ZЁёА-я]+$', t.text) # выкидываем токены не содержащие букв
               or re.match(r'^(\w)\1+$', t.text)  # выкидываем токены из одного повторяющегося символа
               or re.match(r'^[^a-zA-ZЁёА-я].*$', t.text)  # выкидываем токены начинающиеся не с буквы
               or (t.text in stopwords) # выкидываем предлоги, союзы и т.п.    
            )
        ] 

## выполняем частотный анализ

In [7]:
# from sklearn.feature_extraction.text import CountVectorizer
# tf_model = CountVectorizer(
#         min_df=.01, # выкидываем очень редкие слова
#         max_df=.25, # выкидываем очень частые слова
#         tokenizer=tokenizer, # ф-ция токенайзер
#         token_pattern=None, # отключаем дефолтный токенайзер
#         binary=True,
#     )

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_model = TfidfVectorizer(
        min_df=.01, # выкидываем очень редкие слова
        max_df=.10, # выкидываем очень частые слова
        use_idf=False, # не используем обратную частоту
        norm='l2', # нормируем TF
        tokenizer=tokenizer, # ф-ция токенайзер
        token_pattern=None, # отключаем дефолтный токенайзер
        ngram_range = (2,2)
    )

In [9]:
%%time

data_tf = tf_model.fit_transform( data['text'].str.lower() )

display(data_tf.shape)

(3196, 116)

CPU times: user 4.67 s, sys: 61.9 ms, total: 4.73 s
Wall time: 4.76 s


In [10]:
vocab = sorted(tf_model.vocabulary_)
display(len(vocab))
display(vocab)

116

['adobe flash',
 'afisha tut',
 'auto tut',
 'finance tut',
 'flash player',
 'html установлена',
 'javascript ваш',
 'realty tut',
 'sport tut',
 'wall street',
 'ближайшее время',
 'большая часть',
 'браузер поддерживает',
 'ваш браузер',
 'версия проигрывателя',
 'владимир путин',
 'внимание отключен',
 'возбуждено уголовное',
 'вторник декабря',
 'второе место',
 'глава государства',
 'главы государства',
 'говорится сообщении',
 'года назад',
 'данный момент',
 'дек риа',
 'декабря auto',
 'декабря lenta',
 'декабря sport',
 'декабря tut',
 'декабря года',
 'декабря обновлено',
 'декабря тасс',
 'дональд трамп',
 'дональда трампа',
 'друг друга',
 'избранного президента',
 'избранный президент',
 'иностранных дел',
 'конца года',
 'коренных малочисленных',
 'лет назад',
 'лиги чемпионов',
 'лошадиных сил',
 'малочисленных народов',
 'мвд россии',
 'миллиона рублей',
 'миллионов рублей',
 'млн долларов',
 'млн рублей',
 'москва дек',
 'москва декабря',
 'народов севера',
 'настояще

## кластеризируем

In [11]:
# оценки расстояний 
from sklearn.metrics.pairwise import euclidean_distances
d = euclidean_distances(data_tf)
display( d[d>0.].min(),d[d>0.].mean(),d.max(), )

0.018299992189866887

1.2168617334102432

1.4142135623730954

In [12]:
%%time

from sklearn.cluster import DBSCAN
data['cluster_id'] = DBSCAN(eps=.7,min_samples=3).fit(data_tf).labels_
data['cluster_id'].drop_duplicates().count()

CPU times: user 412 ms, sys: 23.9 ms, total: 435 ms
Wall time: 285 ms


83

In [13]:
# номер кластера, количество объектов, метки объектов
# (cluster=-1 - некластеризованные DBSCAN объекты) 
cluster_descr = pd.concat([
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].count(),
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].apply(lambda s: set(s)).apply(' '.join)
    ],axis=1).reset_index()

cluster_descr.columns = ['cluster_id','count','tags']

display( cluster_descr )

,cluster_id,count,tags
0,-1,465,sport politics science tech social incident reclama culture economics realty health woman auto
1,0,921,sport politics science tech social incident culture reclama realty economics health woman auto
2,1,25,sport politics science tech culture economics auto
3,2,17,politics tech social culture economics auto
4,3,120,sport politics science tech social incident culture economics woman auto
...,...,...,...
78,77,10,incident
79,78,5,incident
80,79,3,incident
81,80,3,incident auto


In [14]:
display( data.query('cluster_id==2') )

,text,tag,cluster_id
11,"Путин велел правительству внести в Думу закон об отсрочках... и внести\nв Госдуму проект федерального закона, предусматривающего, в частности,\nпереход с 1 января 2008 года на 12-месячную военную службу по призыву,\nпередает РБК.... ряда оснований для предоставления гражданам отсрочек\nот призыва на военную службу, а также освобождение от призыва на военную\nслужбу студентов, прошедших подготовку на ...",politics,2
18,"Путин поручил внести в Думу проект закона о 12-месячной военной службе...\nРФ доработать и внести в Госдуму законопроект, предусматривающий с 1\nянваря 2008 года переход на 12-месячную военную службу по призыву,\nсообщила пресс-служба Кремля.... также изменение и отмену ряда оснований\nдля предоставления гражданам отсрочек от призыва на военную службу, а\nтакже освобождение от призыва на военную службу студентов, ...",politics,2
351,"Украина предложила России компромисс по цене на газ Комитет Верховной\nРады по вопросам топливно-энергетического комплекса, ядерной политике и\nядерной безопасности предложил новый пакет по закупкам российского газа.\nВ частности, предлагается с 1 января 2006 года покупать 17 миллиардов\nкубометров газа по 50 долларов США, а 8 миллиардов кубометров - по 80-82\nдолларов за тысячу кубометров.",politics,2
830,"В Могилевской области появится своя резиденция Деда Мороза\n\n1 декабря 2016 в 9:27\n\nМогилевский облисполком\n\nВ охотничьем комплексе La Proni 20 декабря откроется «Чаусская резиденция Деда Мороза». В деревне Прилеповке Дед Мороз будет принимать не только детей, но и взрослых — программы для них будут разные.\n\nПринимать и развлекать гостей помимо деда Мороза и Снегурочки будут сказочные персонажи: Кот Баюн, Шуршик и другие. Чтобы попасть в резиденцию, гостям предложат пройти ряд испытаний, для чего вся территория охотничьего комплекса разбита по секторам. А возле волшебного Дуба можно будет загадать желание, чтобы в новом году оно обязательно исполнилось.\n\nВ Чаусской резиденции Дед Мороз будет встречаться с детьми с 10.00 до 18.00. Взрослых ждут с 19.00 до 24.00. Программы и угощения — и стоимость билетов — для этих групп будут различаться.\n\nРезиденция будет работать по предварительным заявкам до 14 января 2017 года.",social,2
1039,"С января 2017 года банки не будут ставить терминалы, которые не принимают бесконтактные карточки\n\n5 декабря 2016 в 12:13\n\nFINANCE.TUT.BY\n\nС 1 января 2017 года белорусские банки перестанут подключать оборудование, которое не поддерживает бесконтактный способ оплаты. Торговцам придется учитывать данный факт при выборе терминального оборудования для своих торговых точек. Об этом говорится в пресс-релизе официального дистрибьютора терминального оборудования Ingenico компании «МВВ-Трейд».\n\nМеждународная платежная система MasterCard в бюллетене № 1 от 15 января 2016 г. объявила, что с 2017 года все новые терминалы оплаты в торговых точках (POS-терминалы) должны устанавливаться с функцией проведения бесконтактных платежей. Аналогичное требование отражено в постановлении Нацбанка от 8 апреля 2016 года.\n\nСегодня на руках у населения более 12,7 миллиона банковских платежных карточек. Из них 1,67 миллиона карт имеют возможность бесконтактного способа оплаты (данные на 1 октября). Пр...",economics,2
1586,"По подсчетам, в среднем он слушал по 67 композиций в день.\n\nВ Санкт-Петербурге живет поклонник творчества группы «Аукцыон», который прослушал песни коллектива 25,3 тыс. раз, а в Новосибирске — фанат Radiohead, который слушал песни британской группы 25,2 тыс. раз. Житель Омска прослушал более 22 тыс. раз песни «Сплина», а Ростова-на-Дону — более 20 тыс. раз группу Metallica, пишет Meduza.\n\nВ топ-10 популярных треков в Краснодарском крае не попала ни одна русскоязычная песня. Возглавляет рейтинг композиция Hymn For The Weekend группы Coldplay. Топ популярных исполнителей возглавили Coldplay, Twenty One Pilots и Юлианна Караулова, сообщили инернет-порталу «Кубань 24» в пресс-службе сервиса.\n\nКак писал интернет порта